# Inicialização

In [6]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

from plotly import graph_objects as go
import plotly as py

from datetime import datetime
from datetime import timedelta

import xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix, accuracy_score, f1_score, confusion_matrix, recall_score, precision_score

from sklearn.utils import resample

# Funções

In [7]:
def upsampleData(X, label):
    # Separate true and false
    false_label = X[X[label]==0].copy()
    true_label = X[X[label]==1].copy()
    
    # Upsample true values
    label_upsampled = resample(true_label,
                            replace=True, # sample with replacement
                            n_samples=len(false_label), # match number in majority class
                            random_state=378) # reproducible results
    upsampled = pd.concat([false_label, label_upsampled])
    
    # Separate x and y
    x = upsampled[[c for c in X.columns if label not in c]]
    y = upsampled[label]
    
    return x, y

In [8]:
def trainXGB(df, cols_rem, label, verbose=True):
    xgb = xgboost.XGBClassifier()

    # Separate x and y and remove unnecessary columns
    x = df[[c for c in df.columns if c not in cols_rem]]
    y = df[label]
    
    # Split training and test data
    x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3, random_state = 378, stratify=y)
    
    # Upsample true values
    X = pd.concat([x_treino, y_treino], axis=1)
    x_treino, y_treino = upsampleData(X, label)

    # XGBClassifier parameters
    param = {'max_depth':50, 'eta':1, 'objective':'binary:logistic', 'min_child_weight': 1, 'lambda': 1, 'alpha': 0, 'gamma': 0}

    # Generate DMatrices with training and test data
    df_train = xgboost.DMatrix(data=x_treino, label=y_treino)
    df_test = xgboost.DMatrix(data=x_teste, label=y_teste)

    # Train model and predict on training and test data
    bst = xgboost.train(param, df_train, 2, feval=f1_score)
    y_teste_pred = bst.predict(xgboost.DMatrix(data=x_teste, label=y_teste))
    y_teste_pred = [1 if i>0.5 else 0 for i in y_teste_pred]
    y_treino_pred = bst.predict(xgboost.DMatrix(data=x_treino, label=y_treino))
    y_treino_pred = [1 if i>0.5 else 0 for i in y_treino_pred]
    
    # Print results if verbose is true
    if verbose:
        print(f"Treino: {accuracy_score(y_treino, y_treino_pred)}")
        print(f"Teste: {accuracy_score(y_teste, y_teste_pred)}")
        print(f"Precisão: {precision_score(y_teste, y_teste_pred)}")
        print(f"Recall: {recall_score(y_teste, y_teste_pred)}")
        print(f"F1: {f1_score(y_teste, y_teste_pred)}")
        display(confusion_matrix(y_teste, y_teste_pred, normalize='true'))
        display(confusion_matrix(y_teste, y_teste_pred,))
        
    # Store results in a dict
    results = {
        'Features': list(x.columns),
        'Train_Acc': accuracy_score(y_treino, y_treino_pred),
        'Test_Acc': accuracy_score(y_teste, y_teste_pred),
        'Precision': precision_score(y_teste, y_teste_pred),
        'Recall': recall_score(y_teste, y_teste_pred),
        'F1': f1_score(y_teste, y_teste_pred),
        'Ver_Pos': confusion_matrix(y_teste, y_teste_pred, normalize='true')[1,1]
    }
    
    return bst, results, y_treino_pred, y_teste_pred

# Prepped Data

In [9]:
df_p = pd.read_csv('../../../data/cleandata/Info pluviometricas/Merged Data/prepped_data.csv', sep=';')
df_p.groupby('Label').count()

,Data_Hora,Mes,Dia,Local,Precipitacao,PrecSum
Label,,,,,,
0.0,359227,359227,359227,359227,359227,359227
1.0,6798,6798,6798,6798,6798,6798


In [10]:
df_p = df_p.sort_values(['Data_Hora', 'Local'])
df_p['Label'] = df_p['Label'].shift(-5*6, fill_value = 0)

In [11]:
# Parameters
label = 'Label'
cols_rem = ['LocalMax', 'Label', 'Label_Old', 'Cluster', 'Data', 'Hora', 'Data_Hora', 'Ordens', 'Minuto'] + [c for c in df_p.columns if 'Hora_' in c]
# Result set
prepped_models = {}

for l in range(6):
    if l != 0:
        df_train = df_p[df_p['Local'] == l]
    else:
        df_train = df_p.copy()
        
    print(f'----- LOCAL {l} -----')
    model, training_res, y_treino_pred, y_teste_pred = trainXGB(df_train, cols_rem, label)
    
    prepped_models[l] = {
        'model': model,
        'results': training_res,
        'y_treino': y_treino_pred,
        'y_teste': y_teste_pred
    }

----- LOCAL 0 -----
Treino: 0.9813408203358016
Teste: 0.9623797901792219
Precisão: 0.3252005347593583
Recall: 0.9543894065718489
F1: 0.4851053221986788


array([[0.96253097, 0.03746903],
       [0.04561059, 0.95438941]])

array([[103731,   4038],
       [    93,   1946]], dtype=int64)

----- LOCAL 1 -----
Treino: 0.9802352660177942
Teste: 0.9577451962480649
Precisão: 0.3102661596958175
Recall: 0.951048951048951
F1: 0.4678899082568807


array([[0.9578786 , 0.0421214 ],
       [0.04895105, 0.95104895]])

...

array([[0.95687332, 0.04312668],
       [0.02477477, 0.97522523]])

array([[20590,   928],
       [   11,   433]], dtype=int64)

----- LOCAL 5 -----
Treino: 0.982269221594975
Teste: 0.9608414534195429
Precisão: 0.31295843520782396
Recall: 0.9576059850374065
F1: 0.47174447174447176


array([[0.96090163, 0.03909837],
       [0.04239401, 0.95760599]])

array([[20718,   843],
       [   17,   384]], dtype=int64)

In [12]:
prepped_models[0]['results']

{'Features': ['Mes', 'Dia', 'Local', 'Precipitacao', 'PrecSum'],
 'Train_Acc': 0.9813408203358016,
 'Test_Acc': 0.9623797901792219,
 'Precision': 0.3252005347593583,
 'Recall': 0.9543894065718489,
 'F1': 0.4851053221986788,
 'Ver_Pos': 0.9543894065718489}

# Full Data

In [13]:
df_f = pd.read_csv('../../../data/cleandata/Info pluviometricas/Merged Data/full_data.csv', sep=';')
display(df_f.head())
df_f.shape

,Data_Hora,LocalMax_d_All,LocalMax_d_ow,Local_d_Null,LocalMax_h_All,LocalMax_h_ow,Local_h_Null,Local,UmidadeRelativa,PressaoAtmosferica,...,TemperaturaInterna,PontoDeOrvalho,RadiacaoSolar,DirecaoDoVento,VelocidadeDoVento,Precipitacao,LocalMax_d,LocalMax_h,Local_d,Local_h
0,2011-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,RM,86.6,926.7,...,23.3,17.8,0.0,131.0,2.5,0.0,0.0,0.0,0.0,0.0
1,2011-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,Erasmo,88.8,920.5,...,22.9,17.7,0.0,133.0,0.7,0.0,0.0,0.0,0.0,0.0
2,2011-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,Camilopolis,84.8,920.6,...,22.8,17.5,0.0,137.0,1.5,0.0,0.0,0.0,0.0,0.0
3,2011-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,Paraiso,92.3,919.1,...,22.7,18.0,0.0,101.0,2.6,0.0,0.0,0.0,0.0,0.0
4,2011-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,Vitoria,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(1464030, 21)

In [14]:
df_f['Data_Hora'] = pd.to_datetime(df_f['Data_Hora'], yearfirst=True)
df_f = df_f[df_f['Data_Hora'].dt.minute == 0]
df_f = df_f.drop(columns = ['LocalMax_d_ow', 'LocalMax_h_All', 'LocalMax_h', 'LocalMax_h_ow', 'LocalMax_d'] + [c for c in df_f.columns if 'Local_' in c])
df_f = df_f.rename(columns = {'LocalMax_d_All': 'Label'})
df_f['Dia'] = df_f['Data_Hora'].dt.day
df_f['Mes'] = df_f['Data_Hora'].dt.month
df_f['Data'] = df_f['Data_Hora'].dt.strftime('%Y-%m-%d')
df_f['Local'] = df_f['Local'].replace({'Camilopolis': 1, 'Erasmo': 2, 'Paraiso': 3, 'RM': 4, 'Vitoria': 5})

In [15]:
df_prec_sum = df_f.groupby(['Data', 'Local']).sum().reset_index()[['Data', 'Local', 'Precipitacao']]
df_prec_sum.columns = ['Data', 'Local', 'PrecSum']
df_f = df_f.merge(df_prec_sum, on=['Data', 'Local'])
df_f.loc[(df_f['Label'] == 1) & (df_f['PrecSum'] <= 10), 'Label'] = 0

In [16]:
cols_dummies = ['Local', 'Mes',]# 'Dia']

df_f_ohe = df_f.copy()

for c in cols_dummies:
    df_f_ohe = pd.concat([df_f_ohe, pd.get_dummies(df_f[c], prefix=c)], axis=1)
    
df_f_ohe = df_f_ohe.sort_values(['Data', 'Local'])

df_f_ohe['Label_Old'] = df_f_ohe['Label']
df_f_ohe['Label'] = df_f_ohe['Label'].shift(-5*6, fill_value = 0)

In [17]:
df_f_ohe.columns

Index(['Data_Hora', 'Label', 'Local', 'UmidadeRelativa', 'PressaoAtmosferica',
       'TemperaturaDoAr', 'TemperaturaInterna', 'PontoDeOrvalho',
       'RadiacaoSolar', 'DirecaoDoVento', 'VelocidadeDoVento', 'Precipitacao',
       'Dia', 'Mes', 'Data', 'PrecSum', 'Local_1', 'Local_2', 'Local_3',
       'Local_4', 'Local_5', 'Mes_1', 'Mes_2', 'Mes_3', 'Mes_4', 'Mes_5',
       'Mes_6', 'Mes_7', 'Mes_8', 'Mes_9', 'Mes_10', 'Mes_11', 'Mes_12',
       'Label_Old'],
      dtype='object')

In [18]:
test_cases = [
    [],
    ['DirecaoDoVento', 'VelocidadeDoVento'],
    ['DirecaoDoVento', 'VelocidadeDoVento', 'TemperaturaInterna'],
    ['DirecaoDoVento', 'VelocidadeDoVento', 'TemperaturaInterna', 'PontoDeOrvalho'],
    ['DirecaoDoVento', 'VelocidadeDoVento', 'TemperaturaInterna', 'PontoDeOrvalho', 'UmidadeRelativa'],
    ['DirecaoDoVento', 'VelocidadeDoVento', 'TemperaturaInterna', 'PontoDeOrvalho', 'UmidadeRelativa', 'PressaoAtmosferica'],
    ['DirecaoDoVento', 'VelocidadeDoVento', 'TemperaturaInterna', 'PontoDeOrvalho', 'UmidadeRelativa', 'PressaoAtmosferica', 'RadiacaoSolar'],
    ['DirecaoDoVento', 'VelocidadeDoVento', 'TemperaturaInterna', 'PontoDeOrvalho', 'UmidadeRelativa', 'PressaoAtmosferica', 'RadiacaoSolar', 'TemperaturaDoAr'],
    ['DirecaoDoVento', 'VelocidadeDoVento', 'TemperaturaInterna', 'PontoDeOrvalho', 'UmidadeRelativa', 'PressaoAtmosferica', 'RadiacaoSolar', 'TemperaturaDoAr', 'Local'],
    ['DirecaoDoVento', 'VelocidadeDoVento', 'TemperaturaInterna', 'PontoDeOrvalho', 'UmidadeRelativa', 'PressaoAtmosferica', 'RadiacaoSolar', 'TemperaturaDoAr', 'PrecSum'],
    ['DirecaoDoVento', 'VelocidadeDoVento', 'TemperaturaInterna', 'PontoDeOrvalho', 'UmidadeRelativa', 'PressaoAtmosferica', 'RadiacaoSolar', 'TemperaturaDoAr', 'Precipitacao']
]

df_training_result = pd.DataFrame(columns = ['Removed_Cols', 'Local', 'Features', 'Train_Acc', 'Test_Acc', 'Precision', 'Recall', 'F1', 'Ver_Pos'])
label = 'Label'

for case in test_cases:
    print(f'---------- CASE ----------')
    print(case)
    print(f'--------------------------')
    for l in range(6):
        if l != 0:
            df_train = df_f_ohe[df_f_ohe['Local'] == l].drop(columns = [c for c in df_f_ohe.columns if 'Local' in c])
        else:
            df_train = df_f_ohe.copy()

        cols_rem = ['Label', 'Label_Old', 'Data', 'Data_Hora'] + cols_dummies
        cols_rem = cols_rem + case
            
        print(f'----- LOCAL {l} -----')
        model, training_res, y_treino_pred, y_teste_pred = trainXGB(df_train, cols_rem, label)
        
        df_training_result = df_training_result.append(
            {**{'Model': model, 'Removed_Cols': case, 'Local': l}, **training_res},
            ignore_index=True
        )

---------- CASE ----------
[]
--------------------------
----- LOCAL 0 -----
Treino: 0.9965700037381988
Teste: 0.9868862013696634
Precisão: 0.6057183198023297
Recall: 0.8415890142226582
F1: 0.7044334975369458


array([[0.98963524, 0.01036476],
       [0.15841099, 0.84158901]])

array([[106652,   1117],
       [   323,   1716]], dtype=int64)

----- LOCAL 1 -----
Treino: 0.9982319168802273
Teste: 0.9911210272288499
Precisão: 0.7031578947368421
Recall: 0.8608247422680413
F1: 0.7740440324449596


array([[0.99346436, 0.00653564],
       [0.13917526, 0.86082474]])

...

array([[0.95342148, 0.04657852],
       [0.02702703, 0.97297297]])

array([[20551,  1004],
       [   11,   396]], dtype=int64)

----- LOCAL 5 -----
Treino: 0.9817147177619929
Teste: 0.9644385757217011
Precisão: 0.3507896924355777
Recall: 1.0
F1: 0.5193846153846153


array([[0.96374188, 0.03625812],
       [0.        , 1.        ]])

array([[20759,   781],
       [    0,   422]], dtype=int64)

In [19]:
#df_training_result.to_csv('../../../data/analysis/training_test_shift.csv', index=False, sep=';', decimal=',')
df_training_result

,Removed_Cols,Local,Features,Train_Acc,Test_Acc,Precision,Recall,F1,Ver_Pos,Model
0,[],0,"[UmidadeRelativa, PressaoAtmosferica, Temperat...",0.996570,0.986886,0.605718,0.841589,0.704433,0.841589,<xgboost.core.Booster object at 0x0000015C4D70...
1,[],1,"[UmidadeRelativa, PressaoAtmosferica, Temperat...",0.998232,0.991121,0.703158,0.860825,0.774044,0.860825,<xgboost.core.Booster object at 0x0000015C4D70...
2,[],2,"[UmidadeRelativa, PressaoAtmosferica, Temperat...",0.996523,0.991075,0.683712,0.925641,0.786492,0.925641,<xgboost.core.Booster object at 0x0000015C4D70...
3,[],3,"[UmidadeRelativa, PressaoAtmosferica, Temperat...",0.998706,0.992441,0.776398,0.866051,0.818777,0.866051,<xgboost.core.Booster object at 0x0000015C4D70...
4,[],4,"[UmidadeRelativa, PressaoAtmosferica, Temperat...",0.995636,0.981559,0.501656,0.744472,0.599407,0.744472,<xgboost.core.Booster object at 0x0000015C4D70...
...,...,...,...,...,...,...,...,...,...,...
61,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...",1,"[Dia, PrecSum, Mes_1, Mes_2, Mes_3, Mes_4, Mes...",0.993861,0.988070,0.596923,1.000000,0.747592,1.000000,<xgboost.core.Booster object at 0x0000015C7081...
62,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...",2,"[Dia, PrecSum, Mes_1, Mes_2, Mes_3, Mes_4, Mes...",0.988278,0.975776,0.422993,1.000000,0.594512,1.000000,<xgboost.core.Booster object at 0x0000015C7081...
63,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...",3,"[Dia, PrecSum, Mes_1, Mes_2, Mes_3, Mes_4, Mes...",0.992515,0.983790,0.548796,1.000000,0.708674,1.000000,<xgboost.core.Booster object at 0x0000015C7081...
64,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...",4,"[Dia, PrecSum, Mes_1, Mes_2, Mes_3, Mes_4, Mes...",0.977602,0.953784,0.282857,0.972973,0.438296,0.972973,<xgboost.core.Booster object at 0x0000015C7081...


In [20]:
df_best_local = pd.DataFrame(columns = df_training_result.columns)

for l in range(6):
    df_best_local = df_best_local.append(df_training_result[(df_training_result['Local'] == l)].sort_values('F1', ascending=False).reset_index(drop=True).loc[0])
    
df_best_local

,Removed_Cols,Local,Features,Train_Acc,Test_Acc,Precision,Recall,F1,Ver_Pos,Model
0,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...",0,"[TemperaturaDoAr, RadiacaoSolar, Precipitacao,...",0.996536,0.990019,0.667853,0.920059,0.773927,0.920059,<xgboost.core.Booster object at 0x0000015C4D6F...
0,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...",1,"[TemperaturaDoAr, RadiacaoSolar, Precipitacao,...",0.998361,0.994582,0.788009,0.948454,0.860819,0.948454,<xgboost.core.Booster object at 0x0000015C4D6F...
0,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...",2,"[PressaoAtmosferica, TemperaturaDoAr, Radiacao...",0.996801,0.993216,0.733075,0.971795,0.835722,0.971795,<xgboost.core.Booster object at 0x0000015C4D6F...
0,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...",3,"[TemperaturaDoAr, Precipitacao, Dia, PrecSum, ...",0.998119,0.995173,0.823762,0.960739,0.886994,0.960739,<xgboost.core.Booster object at 0x0000015C4D6F...
0,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...",4,"[PressaoAtmosferica, TemperaturaDoAr, Radiacao...",0.996610,0.986932,0.614068,0.793612,0.692390,0.793612,<xgboost.core.Booster object at 0x0000015C4D6F...
0,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...",5,"[UmidadeRelativa, PressaoAtmosferica, Temperat...",0.996588,0.988526,0.659176,0.834123,0.736402,0.834123,<xgboost.core.Booster object at 0x0000015C4D6F...


In [21]:
#df_best_local.to_csv('../../../data/analysis/best_shift_local.csv', index=False, sep=';', decimal=',')

# OWM

In [22]:
df_owm = pd.read_csv('../../../data/cleandata/Info pluviometricas/Merged Data/OpenWeatherMapSantoAndre.csv')

In [23]:
df_owm['Data_Hora'] = pd.to_datetime(df_owm['dt_iso'].str[:-10])
df_owm['Data_Hora'] = df_owm.apply(lambda x: x['Data_Hora'] + pd.Timedelta(hours = x['timezone'] / 3600), axis = 1)
df_owm = df_owm[(datetime.strptime('2019-08-30', '%Y-%m-%d') >= df_owm['Data_Hora']) & (df_owm['Data_Hora'] >= datetime.strptime('2010-01-01', '%Y-%m-%d'))]
df_owm = df_owm.drop(columns = ['sea_level', 'grnd_level', 'rain_3h', 'snow_1h', 'snow_3h'])
df_owm = df_owm.fillna(0)
df_owm = df_owm.drop_duplicates(subset='Data_Hora')

In [24]:
df_loc = pd.read_csv('../../../data/cleandata/Ordens de serviço/labels_day.csv', sep=';')
df_loc['Data'] = pd.to_datetime(df_loc['Data'], yearfirst=True)
df_loc = df_loc[['Data', 'LocalMax']]
df_loc.columns = ['Data', 'Label']
df_loc.head()

,Data,Label
0,2010-01-07,1
1,2010-01-08,0
2,2010-01-11,0
3,2010-01-19,0
4,2010-01-20,1


In [25]:
df_owm['Data'] = pd.to_datetime(df_owm['Data_Hora'].dt.strftime('%Y-%m-%d'), yearfirst=True)
df = df_owm.merge(df_loc, on='Data', how='left')
df = df.fillna(0)

In [26]:
df_g = df.groupby('Data').sum().reset_index()[['Data', 'rain_1h']]
df_g.columns = ['Data', 'rain_sum']
df = df.merge(df_g, on='Data')

In [27]:
df['Mes'] = df['Data_Hora'].dt.month
df['Dia'] = df['Data_Hora'].dt.day
# df = df.drop(columns = ['dt', 'dt_iso', 'timezone', 'city_name', 'lat', 'lon', 'weather_icon', 'weather_id', 'weather_main',
#                         'Vitoria', 'Erasmo', 'Paraiso', 'RM', 'Null', 'Camilopolis'])
df = df.drop(columns = ['dt', 'dt_iso', 'timezone', 'city_name', 'lat', 'lon', 'weather_icon', 'weather_id', 'weather_main'])
# df = df.drop(columns = ['dt', 'dt_iso', 'timezone', 'city_name', 'lat', 'lon', 'weather_icon', 'weather_id', 'weather_main',
#                         'Data_Hora', 'Data'])
# df['weather_description'] = df['weather_description'].rank(method='dense', ascending=False).astype(int)

In [28]:
cols_dummies = ['Mes', 'weather_description', 'Dia']

df_ohe = df.copy()

for c in cols_dummies:
    df_ohe = pd.concat([df_ohe, pd.get_dummies(df_ohe[c], prefix=c)], axis=1)
    
df_ohe = df_ohe.sort_values(['Data'])

df_ohe['Label_Old'] = df_ohe['Label']
df_ohe['Label'] = df_ohe['Label'].shift(-1*6, fill_value = 0)

In [29]:
df_ohe.columns

Index(['temp', 'feels_like', 'temp_min', 'temp_max', 'pressure', 'humidity',
       'wind_speed', 'wind_deg', 'rain_1h', 'clouds_all',
       'weather_description', 'Data_Hora', 'Data', 'Label', 'rain_sum', 'Mes',
       'Dia', 'Mes_1', 'Mes_2', 'Mes_3', 'Mes_4', 'Mes_5', 'Mes_6', 'Mes_7',
       'Mes_8', 'Mes_9', 'Mes_10', 'Mes_11', 'Mes_12',
       'weather_description_broken clouds', 'weather_description_drizzle',
       'weather_description_few clouds', 'weather_description_fog',
       'weather_description_haze',
       'weather_description_heavy intensity drizzle',
       'weather_description_heavy intensity rain',
       'weather_description_heavy intesity shower rain',
       'weather_description_light intensity drizzle',
       'weather_description_light intensity drizzle rain',
       'weather_description_light intensity shower rain',
       'weather_description_light rain', 'weather_description_mist',
       'weather_description_moderate rain',
       'weather_description_ov

In [30]:
test_cases = [
    [],
    ['feels_like'],
    ['feels_like', 'temp_min', 'temp_max'],
    ['feels_like', 'temp_min', 'temp_max', 'pressure'],
    ['feels_like', 'temp_min', 'temp_max', 'pressure', 'humidity'],
    ['feels_like', 'temp_min', 'temp_max', 'pressure', 'humidity', 'wind_speed', 'wind_deg'],
    ['feels_like', 'temp_min', 'temp_max', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'clouds_all'],
    ['feels_like', 'temp_min', 'temp_max', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'clouds_all'] + [c for c in df_ohe.columns if 'weather_description' in c],
    ['temp_min', 'temp_max', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'clouds_all'] + [c for c in df_ohe.columns if 'weather_description' in c],
]

df_training_result_owm = pd.DataFrame(columns = ['Removed_Cols', 'Features', 'Train_Acc', 'Test_Acc', 'Precision', 'Recall', 'F1', 'Ver_Pos'])
label = 'Label'

for case in test_cases:
    print(f'---------- CASE ----------')
    print(case)
    print(f'--------------------------')
    
    df_train = df_ohe.copy()

    cols_rem = ['Label', 'Label_Old', 'Data', 'Data_Hora'] + cols_dummies
    cols_rem = cols_rem + case

    model, training_res, y_treino_pred, y_teste_pred = trainXGB(df_train, cols_rem, label)

    df_training_result_owm = df_training_result_owm.append(
        {**{'Removed_Cols': case}, **training_res},
        ignore_index=True
    )

---------- CASE ----------
[]
--------------------------
Treino: 0.9929923158314686
Teste: 0.9678346456692913
Precisão: 0.4446397188049209
Recall: 0.7322720694645442
F1: 0.5533078184800437


array([[0.97442228, 0.02557772],
       [0.26772793, 0.73227207]])

array([[24077,   632],
       [  185,   506]], dtype=int64)

---------- CASE ----------
['feels_like']
--------------------------
Treino: 0.9924892889975889
Teste: 0.9660629921259842
Precisão: 0.42493415276558383
Recall: 0.7004341534008683
F1: 0.5289617486338798


array([[0.97349144, 0.02650856],
       [0.29956585, 0.70043415]])

...

array([[0.97551499, 0.02448501],
       [0.17800289, 0.82199711]])

array([[24104,   605],
       [  123,   568]], dtype=int64)

---------- CASE ----------
['temp_min', 'temp_max', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'clouds_all', 'weather_description', 'weather_description_broken clouds', 'weather_description_drizzle', 'weather_description_few clouds', 'weather_description_fog', 'weather_description_haze', 'weather_description_heavy intensity drizzle', 'weather_description_heavy intensity rain', 'weather_description_heavy intesity shower rain', 'weather_description_light intensity drizzle', 'weather_description_light intensity drizzle rain', 'weather_description_light intensity shower rain', 'weather_description_light rain', 'weather_description_mist', 'weather_description_moderate rain', 'weather_description_overcast clouds', 'weather_description_proximity shower rain', 'weather_description_proximity thunderstorm', 'weather_description_rain and drizzle', 'weather_description_scattered clouds', 'weather_description_shower rain', 'weather_description_sky is clear', 'weather_description_thunderstorm

array([[0.97770043, 0.02229957],
       [0.21997106, 0.78002894]])

array([[24158,   551],
       [  152,   539]], dtype=int64)

In [31]:
df_training_result_owm

,Removed_Cols,Features,Train_Acc,Test_Acc,Precision,Recall,F1,Ver_Pos
0,[],"[temp, feels_like, temp_min, temp_max, pressur...",0.992992,0.967835,0.444640,0.732272,0.553308,0.732272
1,[feels_like],"[temp, temp_min, temp_max, pressure, humidity,...",0.992489,0.966063,0.424934,0.700434,0.528962,0.700434
2,"[feels_like, temp_min, temp_max]","[temp, pressure, humidity, wind_speed, wind_de...",0.992828,0.968228,0.448763,0.735166,0.557323,0.735166
3,"[feels_like, temp_min, temp_max, pressure]","[temp, humidity, wind_speed, wind_deg, rain_1h...",0.993174,0.970551,0.473973,0.751085,0.581187,0.751085
4,"[feels_like, temp_min, temp_max, pressure, hum...","[temp, wind_speed, wind_deg, rain_1h, clouds_a...",0.993478,0.971181,0.480605,0.735166,0.581236,0.735166
5,"[feels_like, temp_min, temp_max, pressure, hum...","[temp, rain_1h, clouds_all, rain_sum, Mes_1, M...",0.992619,0.971929,0.489964,0.777135,0.601007,0.777135
6,"[feels_like, temp_min, temp_max, pressure, hum...","[temp, rain_1h, rain_sum, Mes_1, Mes_2, Mes_3,...",0.992671,0.972165,0.492806,0.793054,0.607876,0.793054
7,"[feels_like, temp_min, temp_max, pressure, hum...","[temp, rain_1h, rain_sum, Mes_1, Mes_2, Mes_3,...",0.991145,0.971339,0.484228,0.821997,0.609442,0.821997
8,"[temp_min, temp_max, pressure, humidity, wind_...","[temp, feels_like, rain_1h, rain_sum, Mes_1, M...",0.992862,0.972323,0.494495,0.780029,0.605278,0.780029


# Teste "real"

In [32]:
df_best_local = df_best_local.reset_index(drop=True)
df_best_local

,Removed_Cols,Local,Features,Train_Acc,Test_Acc,Precision,Recall,F1,Ver_Pos,Model
0,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...",0,"[TemperaturaDoAr, RadiacaoSolar, Precipitacao,...",0.996536,0.990019,0.667853,0.920059,0.773927,0.920059,<xgboost.core.Booster object at 0x0000015C4D6F...
1,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...",1,"[TemperaturaDoAr, RadiacaoSolar, Precipitacao,...",0.998361,0.994582,0.788009,0.948454,0.860819,0.948454,<xgboost.core.Booster object at 0x0000015C4D6F...
2,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...",2,"[PressaoAtmosferica, TemperaturaDoAr, Radiacao...",0.996801,0.993216,0.733075,0.971795,0.835722,0.971795,<xgboost.core.Booster object at 0x0000015C4D6F...
3,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...",3,"[TemperaturaDoAr, Precipitacao, Dia, PrecSum, ...",0.998119,0.995173,0.823762,0.960739,0.886994,0.960739,<xgboost.core.Booster object at 0x0000015C4D6F...
4,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...",4,"[PressaoAtmosferica, TemperaturaDoAr, Radiacao...",0.996610,0.986932,0.614068,0.793612,0.692390,0.793612,<xgboost.core.Booster object at 0x0000015C4D6F...
5,"[DirecaoDoVento, VelocidadeDoVento, Temperatur...",5,"[UmidadeRelativa, PressaoAtmosferica, Temperat...",0.996588,0.988526,0.659176,0.834123,0.736402,0.834123,<xgboost.core.Booster object at 0x0000015C4D6F...


In [33]:
df_m = df_f_ohe[(df_f_ohe['Label'] == 1) | (df_f_ohe['Label_Old'] == 1)].copy()
df_m['Data'] = df_m['Data_Hora'].dt.strftime('%Y-%m-%d')

In [34]:
def getPrecMomento(row):
    prec_momento = df_m.loc[(df_m['Data_Hora'] <= row['Data_Hora']) & (df_m['Local'] == row['Local']) & (df_m['Data'] == row['Data']), 'Precipitacao'].sum()
    return prec_momento

df_m['PrecMomento'] = df_m.apply(getPrecMomento, axis=1)

df_m = df_m.rename(columns = {'PrecSum': 'PrecSumOld', 'PrecMomento': 'PrecSum'})

In [35]:
df_m_2 = df_m.copy()
df_m_2['Label_Pred'] = 0

for l in range(6):
    label_pred = df_best_local.loc[l,'Model'].predict(xgboost.DMatrix(data=df_m_2.loc[df_m_2['Local'] == l, df_best_local.loc[l, 'Features']]))
    df_m_2.loc[df_m_2['Local'] == l, 'Label_Pred'] = [1 if i>0.5 else 0 for i in label_pred]

In [36]:
print(df_m_2[df_m_2['Label_Pred'] == 1].shape)
print(df_m_2.shape)

(2834, 36)
(9848, 36)


In [37]:
df_m_2[['Local', 'Data_Hora', 'Precipitacao', 'PrecSum', 'PrecSumOld', 'Label', 'Label_Pred']].sort_values(by=['Local', 'Data_Hora']
).to_csv('../../../data/analysis/labels_prediction_shift.csv', index=False, sep=';', decimal=',')

In [38]:
pd.set_option("display.max_rows", 200)
df_f_ohe[(df_f_ohe['Data_Hora'] >= datetime(2018,12,23)) & (df_f_ohe['Data_Hora'] <= datetime(2018,12,30)) & (df_f_ohe['Local'] == 4)][['Data_Hora', 'Precipitacao', 'PrecSum']]

,Data_Hora,Precipitacao,PrecSum
343418,2018-12-23 00:00:00,0.0,78.4
343419,2018-12-23 01:00:00,0.0,78.4
343420,2018-12-23 02:00:00,0.0,78.4
343421,2018-12-23 03:00:00,1.0,78.4
343422,2018-12-23 04:00:00,0.6,78.4
343423,2018-12-23 05:00:00,0.0,78.4
343424,2018-12-23 06:00:00,0.0,78.4
343425,2018-12-23 07:00:00,0.0,78.4
343426,2018-12-23 08:00:00,0.0,78.4
343427,2018-12-23 09:00:00,0.0,78.4


In [39]:
df_f_ohe['Local']

48        1
49        1
50        1
51        1
52        1
         ..
365972    5
365973    5
365974    5
365975    5
365976    5
Name: Local, Length: 366025, dtype: int64